In [2]:
import pandas as pd
import json
import numpy as np

```
postprocessing  
intput: df, meta  
output: df  
description:  
하는 것은  
1️⃣ 너무 1초 미만의 등장인물 삭제
2️⃣ 클립  
⭐ 스케일링
```

In [3]:
def postprocessing(df1:pd.DataFrame, meta_info:dict, sec:int=5)->pd.DataFrame:
    # del short length ids, threshold 3sec
    df1 = df1.copy()
    high_rating_id_cnt = 0
    print(f"before filtering id_cnt: {len(df1['track_id'].unique())}")

    for id in df1['track_id'].unique():
        if np.isnan(id): # nan id pass
            continue

        if len(df1[df1['track_id'] == id]) > meta_info['fps']*sec:
            high_rating_id_cnt+=1
        else:
            df1.loc[df1['track_id'] == id,'track_id'] = np.NaN
    
    print(f'after filtering high_rating_id_cnt: {high_rating_id_cnt}')

    # clip
    df1['track_body_xmin'] = df1['track_body_xmin'].clip(0, meta_info['width'])
    df1['track_body_xmax'] = df1['track_body_xmax'].clip(0, meta_info['width'])
    df1['track_body_ymin'] = df1['track_body_ymin'].clip(0, meta_info['height'])
    df1['track_body_ymax'] = df1['track_body_ymax'].clip(0, meta_info['height'])
    
    # scailing
    df1.loc[df1['track_id'].notna(), 'track_body_xmin'] = df1.loc[df1['track_id'].notna(), 'track_body_xmin']/meta_info['width']
    df1.loc[df1['track_id'].notna(), 'track_body_xmax'] = df1.loc[df1['track_id'].notna(), 'track_body_xmin']/meta_info['width']
    df1.loc[df1['track_id'].notna(), 'track_body_ymin'] = df1.loc[df1['track_id'].notna(), 'track_body_xmin']/meta_info['height']
    df1.loc[df1['track_id'].notna(), 'track_body_ymax'] = df1.loc[df1['track_id'].notna(), 'track_body_xmin']/meta_info['height']
    
    postprocessed_df1 = df1
    return postprocessed_df1

```
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

# 원본에 영향을 미치기 때문에 위와같은 오류 문구를 출력함
```

In [6]:
def delete_short_id(df:pd.DataFrame, fps:int, sec:int=5)-> pd.DataFrame:
    df = df.copy()
    for trk_id in df['track_id'].unique():
        # nan id pass
        if np.isnan(trk_id): 
            continue

        # if num of frames over fps*second keep track_id
        if len(df[df['track_id'] == trk_id]) > fps*sec:
            pass
        else:
            df.loc[df['track_id'] == trk_id,'track_id'] = np.NaN
    return df

In [62]:
df

,Unnamed: 0,frame,filename,det_body_xmin,det_body_ymin,det_body_xmax,det_body_ymax,det_conf,track_id,track_body_xmin,track_body_ymin,track_body_xmax,track_body_ymax,track_conf
0,0,1,000001.jpg,330.34960,535.811160,892.01580,1715.1991,0.940190,0.0,330.349609,535.811157,892.015808,1715.199097,0.940190
1,1,1,000001.jpg,2020.36360,529.478300,2466.63670,1712.9662,0.920153,1.0,2020.363647,529.478271,2466.636719,1712.966187,0.920153
2,2,1,000001.jpg,1259.92330,993.807100,1535.52330,1480.1564,0.916883,2.0,1259.923340,993.807129,1535.523315,1480.156372,0.916883
3,3,1,000001.jpg,1294.27110,770.763550,1545.58510,1409.0521,0.896960,3.0,1294.271118,770.763550,1545.585083,1409.052124,0.896960
4,4,2,000002.jpg,334.89594,538.945000,892.85345,1713.9077,0.939980,0.0,334.895935,538.945007,892.853455,1713.907715,0.939980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3184,3184,720,000720.jpg,939.71180,651.762000,1209.87710,1588.4817,0.852101,12.0,939.711792,651.762024,1209.877075,1588.481689,0.852101
3185,3185,720,000720.jpg,1984.10840,380.692020,2448.39920,2084.0657,0.848854,4.0,1984.108398,380.692017,2448.399170,2084.065674,0.848854
3186,3186,720,000720.jpg,605.68365,532.536500,1059.95850,1798.4232,0.845567,8.0,605.683655,532.536499,1059.958496,1798.423218,0.845567
3187,3187,720,000720.jpg,2207.57960,592.927700,2574.61960,1706.9574,0.801158,5.0,2207.579590,592.927673,2574.619629,1706.957397,0.801158


In [110]:
temp = pd.DataFrame([1,2,3,4])
temp["hello"] = [1]*4
temp

,0,hello
0,1,1
1,2,1
2,3,1
3,4,1


```
Index(['frame', 'filename', 'det_body_xmin', 'det_body_ymin', 'det_body_xmax',
       'det_body_ymax', 'det_conf', 'track_id', 'track_body_xmin',
       'track_body_ymin', 'track_body_xmax', 'track_body_ymax', 'track_conf'],
      dtype='object')
```

In [175]:
def count_overlap(df:pd.DataFrame)->pd.DataFrame:
    overlap_list = []
    for frame_num in sorted(df['frame'].unique()): #iter each frame
        overlap_cnt = 0 # counting per id
        one_frame_df = df[df['frame'] == frame_num] # get one frame

        # how to calculate each box relation... just for?
        [1,2],[3,4],[5,6],[7,8]
        for idx, row in one_frame_df.iterrows(): # loop in one frame
            x_min, y_min, x_max, y_max = row['track_body_xmin'], row['track_body_ymin'], row['track_body_xmax'], row['track_body_ymax']
            bbox1, bbox2, bbox3, bbox4, bbox5....



            break
        break
    return df

In [177]:
df = pd.read_csv("/opt/ml/final-project-level3-cv-04/test_PATH/postprocessed_df1.csv", index_col=0)
count_overlap(df)
print()